In [1]:
import pandas as pd
import requests
import os, json
from json import JSONDecodeError

In [2]:
def get_json(from_year, to_year):
    if not 2000 < int(from_year) < 2050 or not 2000 < int(from_year) < 2050:
        return []
    all_holidays = []
    for i in range(from_year, to_year+1):
        r = requests.get(f'https://date.nager.at/api/v2/publicholidays/{i}/CH')
        all_holidays.extend(r.json())
    return all_holidays

def set_date_index(df, col='date'):
    return df.set_index(pd.to_datetime(df[col])).drop(col, axis=1)

def filter_canton(df, can='BS'):
    return df[[(str('CH-' + can) in row) if row is not None else True for row in df.counties]]

def get_holiday_data(canton='BS', from_year=2015, to_year=2019):
    holiday_data = get_json(from_year, to_year)
    holiday_df = (pd.DataFrame.from_records(holiday_data)
                  .pipe(set_date_index)
                  .pipe(filter_canton, can=canton))
    return holiday_df

In [3]:
# 105: Basel
def import_weather_data(station_index=105):    
    stationdata = []
    station_error = 0

    for root, dirs, files in os.walk("weather_data/2018"):
        for file in files: 
            if not file.endswith(".json"): continue
            filename = os.path.join(root, file)
            try:
                with open(filename) as json_file:
                    data = json.load(json_file)
                    stationdata.append(data[station_index])
            except UnicodeDecodeError:
                station_error = station_error + 1
            except JSONDecodeError:
                station_error = station_error + 1
    print ("Loaded: %d - errors: %d" % (len(stationdata), station_error))
    pandata = pd.DataFrame(stationdata)
    pandata = pandata.drop(columns=['code','station'])
    return pandata

def clean_weather_data(weather_df): 
    weather_df['temperature'] = pd.to_numeric(weather_df['temperature'])
    weather_df['sunshine'] = pd.to_numeric(weather_df['sunshine'])
    weather_df['precipitation'] = pd.to_numeric(weather_df['precipitation'])
    return weather_df.set_index(pd.to_datetime(weather_df['dateTime'])).drop('dateTime', axis=1).sort_index()

def get_weather_score():
    weather_data = (import_weather_data()
                    .pipe(clean_weather_data))
    # first version, just take sunshine value
    return weather_data[['sunshine']]

def get_metheo_weather():
    metheo = pd.read_csv('raw_data/metheo.csv', delimiter=';')
    metheo['Date'] = pd.to_datetime(metheo[['Year','Month','Day','Hour','Minute']])
    metheo = metheo.drop(['Year','Month','Day','Hour','Minute'], axis=1).set_index('Date')
    metheo['SWR'] = metheo.pop('Shortwave Radiation')
    return metheo

def get_basel_kanton_weather():
    basel_wetter = pd.read_csv('raw_data/100051.csv', delimiter=';', parse_dates=True)
    basel_wetter.set_index(pd.to_datetime(basel_wetter['Datum/Zeit'])).drop('Datum/Zeit', axis=1).sort_index().dropna()
    return basel_wetter

In [22]:
def import_basel_data():
    data = pd.read_csv("raw_data/Basel_Daten.csv", delimiter=';')
    stamps = [x.split('+')[0] for x in data['DateTimeFrom'].values]
    data = data.assign(Time=pd.to_datetime(stamps.copy(), format="%Y-%m-%dT%H:%M:%S"))
    return data

def clean_location_data(location_df):
    location_df = location_df.set_index('Time')
    location_df = location_df.assign(Day=location_df.index.day)
    return location_df

def get_location_data(df, location):
    mask = df['SiteName'] == location
    return df[mask], df[~mask]

def get_group_data(df, group):
    mask = df['Group'] == group
    return df[mask], df[~mask]

def get_location_split_dict(data):
    location_dict = {}
    #Alle Orte
    locs = sorted(list(set(data['SiteName'])))
    for loc in locs:
        location_df, data = get_location_data(data, loc)
        location_dict[loc] = clean_location_data(location_df)
    return location_dict

def get_group_split_dict(data):
    group_dict = {}
    location_group = {'350 Dreirosenbrücke': 3, '352 Johanniterbrücke': 3, '354 Wettsteinbrücke': 3,
                      '403 Heuwaage-Viadukt': 3, '405 Dorenbachviadukt': 2, '659 Schlachthofstrasse': 3,
                      '660 Flughafenstrasse': 3, '802 Klybeckstrasse 113/Kirche': 1, '803 Johanniterbrücke': 1,
                      '804 Rosentalstrasse 29/28': 2, '805 Rebgasse 11/28': 1, '806 Gerbergasse': 1,
                      '807 Güterstrasse 180/183': 1, '808 Wolfschlucht-Promenade': 1, '809 Allschwilerstrasse 77/86': 1,
                      '810 Neubadstrasse 124/137': 1, '811 Mülhauserstrasse 110/122': 1, '812 Wettsteinbrücke': 1,
                      '813 Hardstrasse 66/77': 1, '814 Elisabethenstrasse 46*': 1, '815 Mittlere Rheinbrücke': 1,
                      '816 Schmiedgasse 4/7 (Riehen)': 1, '817 Elisabethenanlage': 1, '901 Peter-Merian Weg': 3,
                      '902 Viaduktstrasse': 3, '903 Äussere Baselstrasse 328': 2, '904 Hammerstrasse 90': 2,
                      '905 Leimenstrasse 4': 2, '906 Hegenheimerstrasse 44': 2, '907 Wasgenring 62': 2,
                      '908 Grenzacherstrasse (Kraftwerk)': 2, '909 General Guisan-Strasse 104': 2,
                      '910 St. Galler-Ring 101': 2, '911 Birskopfsteg': 2, '912 Elsässerstrasse 261/260': 2,
                      '913 Burgfelderstrasse': 3, '914 Hiltalingerstrasse': 1, '915 Luzernerring-Brücke': 3,
                      '916 Stückisteg': 1, '917 Schwarzwaldbrücke': 3, '918 Elsässerrheinweg': 0,
                      '919 St. Alban-Rheinweg': 2, '920 J. Burckhardt-Strasse': 3}
    data['Group'] = data['SiteName'].apply(lambda x: location_group[x])
    for group in range(4):
        group_df, data = get_group_data(data, group)
        group_dict[group] = clean_location_data(group_df)
    return group_dict

#TODO CREATE REAL SAMPLES
def resample_location_data(location_df, frequency):
    if frequency == 'D':
        cols = ['Total']
    elif frequency == 'H':
        cols = ['Total', 'Month', 'Day', 'Holiday', 'Weekday', 'HourFrom', 'Temperature', 'Precipitation', 'SWR']
    else:
        print("INCORRECT FREQUENCY")
    location_df = location_df[cols].resample(frequency).sum()
    return location_df

In [18]:
holiday_df = get_holiday_data()
#weather_data = (import_weather_data()
#                .pipe(clean_weather_data))[['sunshine', 'precipitation', 'temperature']]
metheo = get_metheo_weather()
data = import_basel_data()
data['Holiday'] = [1 if x in holiday_df.index.values else 0 for x in data['Time']]
data = pd.merge(data, metheo, left_on='Time', right_index=True)

In [23]:
location_hourly_sum = {}
location_daily_sum = {}
for loc, loc_data in get_group_split_dict(data).items():
    location_daily_sum[loc] = resample_location_data(loc_data, 'D')
    location_hourly_sum[loc] = resample_location_data(loc_data, 'H')

In [24]:
location_hourly_sum[1]

,Total,Month,Day,Holiday,Weekday,HourFrom,Temperature,Precipitation,SWR
Time,,,,,,,,,
2015-01-01 00:00:00,23,1,1,0,3,0,-0.86,0.0,0.0
2015-01-01 01:00:00,20,1,1,0,3,1,-0.84,0.0,0.0
2015-01-01 02:00:00,5,1,1,0,3,2,-0.77,0.0,0.0
2015-01-01 03:00:00,5,1,1,0,3,3,-1.36,0.0,0.0
2015-01-01 04:00:00,2,1,1,0,3,4,-1.68,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2019-11-26 19:00:00,41,22,52,0,2,38,18.60,0.0,0.0
2019-11-26 20:00:00,14,22,52,0,2,40,13.76,0.0,0.0
2019-11-26 21:00:00,12,22,52,0,2,42,11.94,0.0,0.0
